### Igneous U-Pb/Hf Data ###
This notebook contains the code used to generate Figure 5 (Bivariate KDEs and crystallization ages of igneous samples) and associated supplementary figures.

In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import rc
rc("pdf", fonttype=42)

from geoscripts.dz import dz

In [ ]:
# Pull new igneous U-Pb/Hf data and create DZ objects

hf_directory = "hf_data/"
samples = ['100311-3A','C17069B','V17043A']

paths = [hf_directory+sample+'.csv' for sample in samples]

data = [pd.read_csv(path) for path in paths]

smps = []
for k,df in enumerate(data):
    smp = dz.DZSample(name=samples[k],agedata=df,source='Vasey22')
    smp.calc_bestage(col_238='Age',col_207='Age')
    smps.append(smp)

In [ ]:
# Pull data from Mayringer

mayringer_directory = os.path.join(hf_directory,'Mayringer11.csv')

mayringer_raw = pd.read_csv(mayringer_directory)

age_cols = ['238Age','238sig2','207Age','207sig2']

# Assume age data for spot 14.2 is equal to spot 14.1 for 345
mayringer_raw.loc[45,age_cols] = mayringer_raw.loc[43,age_cols]

names = list(mayringer_raw['Sample'].unique())
print(names)

In [ ]:

# Calculate crystallization ages for Mayringer samples
msamples = []

for name in names:
    smp_boolean = mayringer_raw.loc[:,'Sample']==name
    data = mayringer_raw.loc[smp_boolean]
    
    sample = dz.DZSample(name=name,agedata=data,source='Mayringer11')
    print(sample.name)
    
    sample.calc_bestage(col_238='238Age',col_207='207Age',
                        err_238='238sig2',err_207='207sig2',err_lev='2sig',
                        filter_disc=False,use_err=True)
    
    young_pop = smp.bestage[(smp.bestage<350)].count()
    
    sample.syst_238 = 0.7

    sample.calc_mda(method='manual',grains=np.arange(1,young_pop,1),systematic=True)

    msamples.append(sample)

In [ ]:
# Calculate crystallization ages for new samples

dat_path = os.path.join(hf_directory,'Vasey_Upb9-19_datatable.xls')
data = pd.read_excel(dat_path,skiprows=22)

age_columns = ['Analysis','Best age','±.6']
agedata = data[age_columns].dropna(how='any')
#print(agedata)

age_cutoffs = [190,190,330]

syst_238 = 1.063742485

ig_upb = []
samples=['100311-3A','C17069B','V17043A']

for k,name in enumerate(samples):
    ages = agedata[agedata.Analysis.str.contains(name)]
    ages.loc[:,'±.6'] = ages.loc[:,'±.6']*2
    smp = dz.DZSample(name=name,agedata=ages)

    smp.calc_bestage(col_238='Best age',col_207='Best age',err_238='±.6',err_207='±.6',err_lev='1sig',
                        filter_disc=False,use_err=True)
    smp.convert_1sigto2sig()
    print(smp.error_level)
    
    young_pop = smp.bestage[smp.bestage<age_cutoffs[k]].count()
    print(young_pop)

    smp.syst_238 = syst_238

    mda = smp.calc_mda(method='manual',grains=np.arange(0,young_pop,1),overdisperse=True,systematic=True)
    ig_upb.append(smp)

    print(mda[2],mda[3])


In [ ]:
# Plot bivariate KDEs for figure

fig,axs = plt.subplots(1,3,dpi=300,figsize=(6.5,2.5))

age_cutoffs = [190,190,330]
title_text = ['SVAN (Andesite)','SVGD (Granodiorite)','SVGR (Granite)']

for k,smp in enumerate(smps):
    smp.kde_hf('Ehf',method=[5,1.5],cmap='Reds',xlim=(100,400),ylim=(-15,15),ax=axs.flat[k],vmin=0.05,include_ages=True,
    marker_color='firebrick',alpha=1)
    axs.flat[k].xaxis.label.set_size(8)
    axs.flat[k].xaxis.label.set_size(8)
    axs.flat[k].set_xticks([100,200,300,400])
    axs.flat[k].set_yticks(np.arange(-15,16,5))
    axs.flat[k].set_xticklabels([100,200,300,400],fontsize=6)
    axs.flat[k].set_yticklabels(np.arange(-15,16,5),fontsize=6)
    axs.flat[k].set_title(title_text[k],fontsize=8)

    age,err = ig_upb[k].mda,ig_upb[k].mda_err

    hf_young = smp.agedata[smp.agedata['Age']<age_cutoffs[k]]

    mean_hf = np.mean(hf_young['Ehf'])
    hf2sig = hf_young['Ehf'].std()*2
    
    hf_text = (
        'Cryst. Age:\n' + str(round(age,1)) + ' ± ' +  str(round(err,1)) +
        ' Ma\n$\epsilon Hf_i$: ' + str(round(mean_hf,1)) + ' ± ' + str(round(hf2sig,1))
    )
    axs.flat[k].text(0.05,0.8,hf_text,transform=axs.flat[k].transAxes,fontsize=6)

for smp in [msamples[1]]:
    hf_young = smp.agedata[(smp.agedata['238Age']<350)&(smp.agedata['238Age']>300)]

    mean_hf = np.mean(hf_young['epsHfinit_rep'])
    hf2sig = hf_young['epsHfinit_rep'].std()*2
    hf_text = (
        'GE0345\nCryst. Age: ' + str(round(smp.mda,1)) + ' ± ' +  str(round(smp.mda_err,1)) + 
        ' Ma\n$\epsilon Hf_i$: ' + str(round(mean_hf,1)) + ' ± ' + str(round(hf2sig,1))
    )
    axs.flat[2].text(0.05,0.05,hf_text,transform=axs.flat[k].transAxes,fontsize=6)

    smp.kde_hf('epsHfinit_rep',method=[5,1.5],cmap='Blues',xlim=(100,400),ylim=(-15,15),ax=axs.flat[2],vmin=0.05,include_ages=True,
    marker_color='skyblue',alpha=0.7)


axs.flat[2].legend(fontsize=6)

plt.tight_layout()
fig.savefig('hf.pdf')


In [ ]:
# Plot and save figure of crystallization ages for supplement

samples = ig_upb + [msamples[1]]

fig, axs = plt.subplots(2,2,dpi=300,figsize=(6.5,4.5))

for k,smp in enumerate(samples):
    print(smp.name,smp.mda)
    smp.plot_mda(axs.flat[k],syst_error=True)

plt.tight_layout()

fig.savefig('cryst_ages.pdf')